In [1]:
# Step 0a: Load packages

# Import the xarray package
import xarray as xr
# Iport the numpy package
import numpy as np
# Import the math package
import math
# Import the module pyplot from the package matplotlib
import matplotlib.pyplot as plt
# Import the package os (operating system interface).
import os
# Import the cartopy package for plotting the coastline
import cartopy.crs as ccrs
# Import features of cartopy for plotting country borders
import cartopy.feature as cf
# Import datetime package for temporal interpolation
import pandas as pd
# Import widgets for interactive usage
import ipywidgets as ipyw
from IPython.display import display

# Import intake package
import intake
# Import easy gems with healpix
from easygems import healpix as egh

In [2]:
# Global variables
pathEarthCare = '/work/mh0731/m301196/ecomip/ftp.eorc.jaxa.jp/eorc/CPR/1B/xCa/2024/08/01/'
engineType = 'netcdf4'
plotEarthCare = True

In [3]:
def readPath(fileName):

    filePath = pathEarthCare + fileName

    var = xr.open_dataset(filePath, engine=engineType)

    lon = np.array(var['lon']).astype(np.float64)
    lat = np.array(var['lat']).astype(np.float64)

    return lon, lat

In [4]:
def readVariable(variableName, fileName):

    filePath = pathEarthCare + fileName

    var = xr.open_dataset(filePath, engine=engineType)

    variable = np.array(var[variableName]).astype(np.float64)

    return variable

In [5]:
def plotEarthCarePath(lon, lat):
    # Set up a figure with Plate Carrée (rectangular) projection
    fig = plt.figure(figsize=(10, 5))
    ax = plt.axes(projection=ccrs.PlateCarree())

    # Add features: coastlines, borders, etc.
    ax.coastlines()
    ax.add_feature(cf.BORDERS, linestyle=':')
    ax.gridlines(draw_labels=True)

    ax.set_xlim(-180, 180)
    ax.set_ylim(-90, 90)

    # Add a title
    plt.title('EarthCare path')

    # Plot the points
    ax.plot(lon, lat, 'ro', markersize=5, transform=ccrs.PlateCarree())

    print(lon)
    print(lat)
    
    # Show the plot
    plt.show()

In [6]:
def plotVariable(variableName, var):
    # Add a title
    title = variableName
    plt.title(title)

    # Plot using indices as axes
    plt.imshow(var.T, origin='lower', cmap='viridis', aspect='auto')

    plt.gca().invert_yaxis()
        
    # Add colorbar
    plt.colorbar(label='Value')

    # Add axis labels and title
    plt.xlabel('Column Index')
    plt.ylabel('Row Index')
    plt.title('Plot curtain for ' + variableName)

    # Show the plot
    plt.show()

In [7]:
def plotVariableOverHeight(variableName, var, heights, lons):
    # Add a title
    title = variableName
    plt.title(title)

    # Plot the points
    height = heights.flatten()
    values = var.flatten()
    lon = np.zeros(len(values))

    for j in range(var.shape[0]):
        for k in range(var.shape[1]):
            lon[j * var.shape[1] + k] = lons[j]
    
    mask = np.isfinite(lon) & np.isfinite(height) & np.isfinite(values) & (height < 15000)

    lon_clean = lon[mask]
    height_clean = height[mask]
    values_clean = values[mask]
        
    # Plot using indices as axes
    plt.tricontourf(lon_clean, height_clean, values_clean, origin='lower', cmap='viridis')
        
    # Add colorbar
    plt.colorbar(label='Value')

    # Add axis labels and title
    plt.xlabel('longitude')
    plt.ylabel('height')
    plt.title('Plot curtain for ' + variableName)

    # Show the plot
    plt.show()

In [8]:
def plotEarthCareData(fileName, heights, lons, lats):

    outputB = ipyw.Output()
    #######################################################################################################################
    # Function to handle visualizations based on selections (is executed after pressing the button)
    def on_buttonB_click(b):

        ###################################################################################################################
        # Continue only if the some selection was made in the dropdown menu
        with outputB:

            ###########################################################################################################
            # Clear previous outputs
            outputB.clear_output()
            
            ###########################################################################################################
            # Retrieving the values from the widgets and store them in global variables
            varName = dropdownB.value

            ###########################################################################################################
            # Call the following function to continue
            variables = readVariable(varName, fileName)
            plotVariableOverHeight(varName, variables, heights, lons)

    filePath = pathEarthCare + fileName

    var = xr.open_dataset(filePath, engine=engineType)

    varNames = [str(v) for v in var if str(v) != 'lon' and str(v) != 'lat' and str(v) != 'height']
    
    dropdownB = ipyw.Dropdown(
        options=varNames,
        description='Variable selection:   ',
        style={'description_width': 'initial'},
        indent=False,
        layout=ipyw.Layout(width='35%', margin='30px 100px 30px 0px')
    )

    display(dropdownB)

    #######################################################################################################################
    # Button to trigger computation
    buttonB = ipyw.Button(description="Confirm selection", layout=ipyw.Layout(margin='30px 0px 30px 0px'))
    buttonB.on_click(on_buttonB_click)

    display(buttonB)

    display(outputB)

In [9]:
def plotSimulationData(fileName, heights, lons, lats):

    outputB = ipyw.Output()
    #######################################################################################################################
    # Function to handle visualizations based on selections (is executed after pressing the button)
    def on_buttonB_click(b):

        ###################################################################################################################
        # Continue only if the some selection was made in the dropdown menu
        with outputB:

            ###########################################################################################################
            # Clear previous outputs
            outputB.clear_output()
            
            ###########################################################################################################
            # Retrieving the values from the widgets and store them in global variables
            varName = dropdownB.value

            ###########################################################################################################
            # Call the following function to continue
            variables = readVariable(varName, fileName)
            plotVariableOverHeight(varName, variables, heights, lons)

    filePath = pathEarthCare + fileName

    var = xr.open_dataset(filePath, engine=engineType)

    varNames = [str(v) for v in var if str(v) != 'lon' and str(v) != 'lat' and str(v) != 'height']
    
    dropdownB = ipyw.Dropdown(
        options=varNames,
        description='Variable selection:   ',
        style={'description_width': 'initial'},
        indent=False,
        layout=ipyw.Layout(width='35%', margin='30px 100px 30px 0px')
    )

    display(dropdownB)

    #######################################################################################################################
    # Button to trigger computation
    buttonB = ipyw.Button(description="Confirm selection", layout=ipyw.Layout(margin='30px 0px 30px 0px'))
    buttonB.on_click(on_buttonB_click)

    display(buttonB)

    display(outputB)

In [12]:
# Start the visualization script by filling out the basic setup
# Create an Output widget to display the results
outputA = ipyw.Output()

#######################################################################################################################
# Function to handle visualizations based on selections (is executed after pressing the button)
def on_buttonA_click(b):

    ###################################################################################################################
    # Continue only if the some selection was made in the dropdown menu
    with outputA:
        ###########################################################################################################
        # Retrieving the values from the widgets and store them in global variables
        pathFile = dropdown.value

        ###########################################################################################################
        # Clear previous outputs
        outputA.clear_output()

        ###########################################################################################################
        # Call the following function to continue with the setup
        lonCoord, latCoord = readPath(pathFile)
        plotEarthCarePath(lonCoord, latCoord)
        heightCoord = readVariable('height', pathFile)
        if plotEarthCare:
            plotEarthCareData(pathFile, heightCoord, lonCoord, latCoord)
        else:
            plotSimulationData(pathFile, heightCoord, lonCoord, latCoord)

# List all files and folders
all_entries = os.listdir(pathEarthCare)

# Filter only files
files = [f for f in all_entries if os.path.isfile(os.path.join(pathEarthCare, f))]

dropdown = ipyw.Dropdown(
    options=files,
    description='Path selection:   ',
    style={'description_width': 'initial'},
    indent=False,
    layout=ipyw.Layout(width='35%', margin='30px 100px 30px 0px')
)

display(dropdown)

#######################################################################################################################
# Button to trigger computation
buttonA = ipyw.Button(description="Confirm selection", layout=ipyw.Layout(margin='30px 0px 30px 0px'))
buttonA.on_click(on_buttonA_click)

display(buttonA)

# Display the output widget to show the results
display(outputA)

Dropdown(description='Path selection:   ', layout=Layout(margin='30px 100px 30px 0px', width='35%'), options=(…

Button(description='Confirm selection', layout=Layout(margin='30px 0px 30px 0px'), style=ButtonStyle())

Output()